In [2]:
# %pip install pandas
# %pip install openpyxl
# %pip install pyreadr

In [3]:
import pandas as pd
import numpy as np
import openpyxl
import re
import pyreadr
import random
# import seaborn as sns
# import matplotlib.pyplot as plt
import string
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

In [4]:
################################################################
#  Import only 1M random rows from voter file
################################################################

random.seed(481516)

total_rows = sum(1 for _ in open('voter_file.csv')) - 1
print(total_rows)

n_sample = 1000000

# Generate a random sample of row indices as a set
random_indices = sorted(np.random.choice(total_rows, n_sample, replace=False))
random_indices_set = set(random_indices)

def skip_rows(row):
    if row == 0:
        return False  # Keep the header
    return (row - 1) not in random_indices_set

vf = pd.read_csv('voter_file.csv', skiprows=lambda x: skip_rows(x))
print(vf.head())

4258869


/tmp/ipykernel_635/2086998383.py:21: DtypeWarning: Columns (45,46,47,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  vf = pd.read_csv('voter_file.csv', skiprows=lambda x: skip_rows(x))


      voterbase_id  version                 household_id_hash  \
0  MO-000000000272  2024_01  1d377757748fe79a8a1a4cf522cae8c6   
1  MO-000000000540  2024_01  39425122af1b943b6208776e4aa244b1   
2  MO-000000000555  2024_01  539c8bd38ce740a91f92e03425c145f5   
3  MO-000000000558  2024_01  f7512941cd96f66cd99a9270fd44d06e   
4  MO-000000000559  2024_01  80e660e23c1d641e455d254f1721e1d5   

  tsmart_exact_track_id  smartvan_id tsmart_precinct_id    sos_id  \
0       Y29454024536170        162.0                 07  16838839   
1       Y29458843277590        364.0                 01  15547155   
2       Y29454759950115        374.0                 01  25665345   
3       Y29454694517123        377.0                 03  16710338   
4       Y29454809304855        378.0            CLA.022  73539213   

   legal_commercial_model_usage_ok  vb_deceased  vote_g2024  ...  \
0                                1            0           0  ...   
1                                1            0           

In [5]:
# Import voter file
# vf = pd.read_csv('voter_file.csv')

# Import Jacobson-Carson
result = pyreadr.read_r('house_election_data_46_22 copy.rds')
df = result[None]
# df.to_csv('all_data.csv', index=False)

print(vf.shape[0])

1000000


In [6]:
################################################################
#  Clean J-C
################################################################

# Convert all columns to integers except for the exclude_columns

exclude_columns = ["dv", "dvp", "dpres"]

for column in df.columns:
    if column not in exclude_columns:
        df[column] = df[column].astype(float).fillna(0).astype(int)

# print(df.head())

# Turn negative values into NaNs

df = df.mask(df < 0, np.nan)
# print(df.head())
# print(len(df))

# Remove any rows where 'pwin' = NaN

df.dropna(subset=['pwin'], inplace=True)
# print(len(df))

# Split "stcd" into state and district

def split_stcd(stcd):
    stcd_str = str(stcd)
    if len(stcd_str) == 3:
        state = int(stcd_str[0])
        dist = int(stcd_str[1:])
    elif len(stcd_str) == 4:
        state = int(stcd_str[:2])
        dist = int(stcd_str[2:])
    else:
        raise ValueError("Invalid length of stcd")
    return state, dist

# Apply the function to split the stcd column
df[['state', 'dist']] = df['stcd'].apply(split_stcd).apply(pd.Series)

# Turn real variables into ranges using maps

def map_exp_granular(value):
    if value <= 50000:
        return 0
    elif value <= 100000:
        return 1
    elif value <= 200000:
        return 2
    elif value <= 300000:
        return 3
    elif value <= 400000:
        return 4
    elif value <= 500000:
        return 5
    elif value <= 600000:
        return 6
    elif value <= 700000:
        return 7
    elif value <= 800000:
        return 8
    elif value <= 900000:
        return 9
    elif value <= 1000000:
        return 10
    elif value <= 1250000:
        return 11
    elif value <= 1500000:
        return 12
    elif value <= 1750000:
        return 13
    elif value <= 2000000:
        return 14
    elif value <= 2500000:
        return 15
    elif value <= 3000000:
        return 16
    elif value <= 4000000:
        return 17
    elif value <= 5000000:
        return 18
    elif value <= 6000000:
        return 19
    elif value <= 7500000:
        return 20
    elif value <= 9000000:
        return 21
    elif value <= 12000000:
        return 22
    elif value <= 15000000:
        return 23
    elif value <= 17500000:
        return 24
    elif value <= 20000000:
        return 25
    elif value <= 22500000:
        return 26
    elif value <= 25000000:
        return 27
    else: pass

def map_exp(value):
    if value <= 100000:
        return 0
    elif value <= 400000:
        return 1
    elif value <= 600000:
        return 2
    elif value <= 900000:
        return 3
    elif value <= 1250000:
        return 4
    elif value <= 1500000:
        return 5
    elif value <= 2000000:
        return 6
    elif value <= 3000000:
        return 7
    elif value <= 6000000:
        return 8
    elif value <= 9000000:
        return 9
    elif value <= 12000000:
        return 10
    elif value <= 17500000:
        return 11
    elif value <= 22500000:
        return 12
    else: pass

df['dexp_cat_gran'] = df['dexp'].apply(map_exp_granular)
df['rexp_cat_gran'] = df['rexp'].apply(map_exp_granular)

df['dexp_cat'] = df['dexp'].apply(map_exp)
df['rexp_cat'] = df['rexp'].apply(map_exp)

def map_voteshare(value):
    if value <= 10:
        return 0
    elif value <= 15:
        return 1
    elif value <= 20:
        return 2
    elif value <= 25:
        return 3
    elif value <= 30:
        return 4
    elif value <= 33:
        return 5
    elif value <= 36:
        return 6
    elif value <= 39:
        return 7
    elif value <= 41:
        return 8
    elif value <= 43:
        return 9
    elif value <= 45:
        return 10
    elif value <= 46:
        return 11
    elif value <= 47:
        return 12
    elif value <= 48:
        return 13
    elif value <= 49:
        return 14
    elif value <= 49.5:
        return 15
    elif value <= 50:
        return 16
    elif value <= 50.5:
        return 17
    elif value <= 51:
        return 18
    elif value <= 52:
        return 19
    elif value <= 53:
        return 20
    elif value <= 54:
        return 21
    elif value <= 55:
        return 22
    elif value <= 57:
        return 23
    elif value <= 59:
        return 24
    elif value <= 61:
        return 25
    elif value <= 64:
        return 26
    elif value <= 67:
        return 27
    elif value <= 70:
        return 28
    elif value <= 75:
        return 29
    elif value <= 80:
        return 30
    elif value <= 85:
        return 31
    elif value <= 90:
        return 32
    elif value <= 100:
        return 33
    else: pass

df['dpres_cat'] = df['dpres'].apply(map_voteshare)
df['dvp_cat'] = df['dvp'].apply(map_voteshare)
df['dv_cat'] = df['dv'].apply(map_voteshare)

# Delete original columns
del df['dvp'], df['dpres'], df['dexp'], df['rexp'], df['stcd'], df['dv']

# print(df.head())

# df.to_csv('jacobson_data.csv', index=False)

MO_recent = df[(df['year'] >= 2012) & (df['state'].isin([4, 14, 13, 16, 25, 27, 42]))]
print(MO_recent.head())
print(MO_recent['dist'].unique())

       year  inc  pwin  fr  po1  po2  redist  state  dist  dexp_cat_gran  \
14375  2012    0   0.0   1  1.0  0.0       1      4     1            4.0   
14376  2012    0   0.0   1  0.0  0.0       1      4     2            3.0   
14377  2012    0   0.0   1  5.0  2.0       1      4     3            NaN   
14378  2012    2   0.0   9  2.0  5.0       1      4     4            1.0   
14490  2012    1   1.0   0  1.0  0.0       1     13     1            5.0   

       rexp_cat_gran  dexp_cat  rexp_cat  dpres_cat  dvp_cat  dv_cat  
14375           12.0       1.0       5.0        7.0     11.0     9.0  
14376           13.0       1.0       6.0       10.0      8.0     9.0  
14377            7.0       NaN       3.0        5.0      4.0     NaN  
14378           15.0       0.0       7.0        7.0     24.0     7.0  
14490            1.0       2.0       0.0       30.0     31.0    29.0  
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]


In [7]:
################################################################
#  Clean vf
################################################################

# Delete irrelevant columns

del vf['version'], vf['household_id_hash'], vf['tsmart_exact_track_id'], vf['smartvan_id'], vf['tsmart_precinct_id'], vf['sos_id'], vf['tsmart_census_block_fips_2010'], vf['tsmart_cd_version_2020'], vf['tsmart_cd_2020'], vf['tsmart_cd_version_2010'], vf['tsmart_sd'], vf['tsmart_sd_2010']
del vf['tsmart_sd_version'], vf['tsmart_sd_version_2010'], vf['tsmart_hd_version'], vf['tsmart_hd_version_2020'], vf['tsmart_hd_version_2010'], vf['vf_reg_cd_version'], vf['vf_reg_cd_version_2010'], vf['vf_reg_sd_2010'], vf['vf_reg_sd_version'], vf['vf_reg_sd_version_2020'], vf['vf_reg_sd_version_2010'], vf['vf_reg_hd'], vf['vf_reg_hd_2010']
del vf['vf_reg_hd_version_2010'], vf['vf_reg_census_id'], vf['vf_reg_census_id_2010'], vf['vf_reg_census_id_version'], vf['vf_reg_census_id_version_2010'], vf['vf_reg_precinct_name'], vf['vf_reg_place_name'], vf['vb_dob'], vf['vote_any_primary_since_2012'], vf['tsmart_census_id_version_2010']

############ del vf['turnout_score'] # Decide whether to include this or not

# Create columns to be aligned with J-C

vf['vf_reg_state'] = vf['vf_reg_dma_name'].str[-2:]

# print(df['vf_reg_state'].unique())
# print(df['vf_reg_cd'].unique())

state_order = {'AR': 4, 'IA': 14, 'IL': 13, 'KS': 16, 'MO': 25, 'NE': 27, 'TN': 42}
vf['vf_state_num'] = vf['vf_reg_state'].map(state_order)

# print(df['vf_state_num'].unique())

# Normalize scores

vf['gotv_score'] = (vf['gotv_score'] - np.min(vf['gotv_score'])) / (np.max(vf['gotv_score']) - np.min(vf['gotv_score']))
vf['synthetic_persuasion_score'] = (vf['synthetic_persuasion_score'] - np.min(vf['synthetic_persuasion_score'])) / (np.max(vf['synthetic_persuasion_score']) - np.min(vf['synthetic_persuasion_score']))

def map_scores(value):
    if value <= .10:
        return 0
    elif value <= .20:
        return 1
    elif value <= .30:
        return 2
    elif value <= .36:
        return 3
    elif value <= .41:
        return 4
    elif value <= .45:
        return 5
    elif value <= .47:
        return 6
    elif value <= .49:
        return 7
    elif value <= .51:
        return 8
    elif value <= .53:
        return 9
    elif value <= .55:
        return 10
    elif value <= .59:
        return 11
    elif value <= .64:
        return 12
    elif value <= .70:
        return 13
    elif value <= .75:
        return 14
    elif value <= .80:
        return 15
    elif value <= .90:
        return 16
    elif value <= 1.00:
        return 17
    else: pass

vf['gotv_score'] = vf['gotv_score'].apply(map_scores)
vf['turnout_score'] = vf['turnout_score'].apply(map_scores)
vf['support_score'] = vf['support_score'].apply(map_scores)
vf['synthetic_persuasion_score'] = vf['synthetic_persuasion_score'].apply(map_scores)

print(vf.head())

      voterbase_id  legal_commercial_model_usage_ok  vb_deceased  vote_g2024  \
0  MO-000000000272                                1            0           0   
1  MO-000000000540                                1            0           0   
2  MO-000000000555                                1            0           0   
3  MO-000000000558                                1            0           0   
4  MO-000000000559                                1            0           0   

   vote_g2023  vote_g2022  vote_g2021  vote_g2020  vote_g2019  vote_g2018  \
0           0           1           0           1           0           0   
1           0           1           0           1           0           1   
2           0           1           0           1           0           1   
3           0           1           0           1           0           1   
4           0           1           0           1           0           1   

   ...  home_value_bucket  household_net_worth_bucket  i

In [8]:
################################################################
#  Add year-specific columns to J-C
################################################################

MO_recent = MO_recent.copy()

columns_to_extend = ['fr', 'inc', 'po1', 'po2', 'redist', 'rexp_cat_gran', 'dexp_cat', 'rexp_cat', 'dpres_cat', 'dvp_cat', 'dexp_cat_gran']

# Loop through each unique year in 'MO_recent'
for yr in MO_recent['year'].unique():

    for col in columns_to_extend:
        
        column_name = f'{col}_{yr}'
        MO_recent.loc[:, column_name] = np.nan
        MO_recent.loc[MO_recent['year'] >= yr, column_name] = MO_recent.loc[MO_recent['year'] >= yr, col]

    pwin_col = f'pwin_{yr}'
    MO_recent.loc[:, pwin_col] = np.nan

    # Choose to leave pwin in or out by changing >= to > below
    MO_recent.loc[MO_recent['year'] >= yr, pwin_col] = MO_recent.loc[MO_recent['year'] >= yr, 'pwin']

# Add information about presidential candidates for election years

categories = ['inc', 'fr', 'po1', 'po2']

values = {
    2012: {'inc': 1, 'fr': 0, 'po1': 1, 'po2': 0},
    2016: {'inc': 2, 'fr': 0, 'po1': 1, 'po2': 0},
    2020: {'inc': 0, 'fr': 0, 'po1': 1, 'po2': 0},
    2024: {'inc': 1, 'fr': 0, 'po1': 1, 'po2': 0},
}

years = list(MO_recent['year'].unique()) + [2024]

# Loop through each year in the range
for yr in years:
    if yr in values:
        for cat in categories:
            # Create the column name 'pres_cat_xxxx' where 'cat' is replaced with the category and 'xxxx' is replaced with the year
            column_name = f'pres_{cat}_{yr}'
            
            # Add the new column to the DataFrame, initializing with NaN using .loc
            MO_recent.loc[:, column_name] = values[yr][cat]
            
            # Set the entries of 'pres_cat_xxxx' to the values defined for the specific year
            # MO_recent.loc[MO_recent['year'] == yr, column_name] = values[yr][cat]

# Create MO_subset from MO_recent
columns_to_exclude = ['inc', 'pwin', 'fr', 'po1', 'po2', 'redist', 
                      'dexp_cat_gran', 'rexp_cat_gran', 'dexp_cat', 'rexp_cat', 
                      'dpres_cat', 'dvp_cat', 'dv_cat']

MO_subset = MO_recent.drop(columns=columns_to_exclude)

# Print the DataFrame to see the changes
print(MO_subset.head())

       year  state  dist  fr_2012  inc_2012  po1_2012  po2_2012  redist_2012  \
14375  2012      4     1      1.0       0.0       1.0       0.0          1.0   
14376  2012      4     2      1.0       0.0       0.0       0.0          1.0   
14377  2012      4     3      1.0       0.0       5.0       2.0          1.0   
14378  2012      4     4      9.0       2.0       2.0       5.0          1.0   
14490  2012     13     1      0.0       1.0       1.0       0.0          1.0   

       rexp_cat_gran_2012  dexp_cat_2012  ...  pres_po1_2016  pres_po2_2016  \
14375                12.0            1.0  ...              1              0   
14376                13.0            1.0  ...              1              0   
14377                 7.0            NaN  ...              1              0   
14378                15.0            0.0  ...              1              0   
14490                 1.0            2.0  ...              1              0   

       pres_inc_2020  pres_fr_2020  pres_po1

In [9]:
################################################################
#  To get most info, subset for only most recent year
################################################################

MO_2022 = MO_subset[MO_subset['year'] == 2022]
print(MO_2022.head())
# for col in MO_2022.columns:
#     print(col)

       year  state  dist  fr_2012  inc_2012  po1_2012  po2_2012  redist_2012  \
16550  2022      4     1      0.0       0.0       1.0       4.0          1.0   
16551  2022      4     2      0.0       0.0       0.0       0.0          1.0   
16552  2022      4     3      0.0       0.0       0.0       0.0          1.0   
16553  2022      4     4      0.0       0.0       0.0       0.0          1.0   
16666  2022     13     1      9.0       2.0       0.0       0.0          1.0   

       rexp_cat_gran_2012  dexp_cat_2012  ...  pres_po1_2016  pres_po2_2016  \
16550                 7.0            0.0  ...              1              0   
16551                16.0            0.0  ...              1              0   
16552                11.0            0.0  ...              1              0   
16553                12.0            0.0  ...              1              0   
16666                 0.0            3.0  ...              1              0   

       pres_inc_2020  pres_fr_2020  pres_po1

In [10]:
################################################################
#  Merge vf and J-C (most recent) before centering around years
################################################################

vf_merged = vf.merge(MO_2022, left_on=['vf_state_num', 'vf_reg_cd'], right_on=['state', 'dist'], how='left')
vf_merged = vf_merged.drop(columns=['state', 'dist', 'year'])
for col in vf_merged.columns:
    print(col)

voterbase_id
legal_commercial_model_usage_ok
vb_deceased
vote_g2024
vote_g2023
vote_g2022
vote_g2021
vote_g2020
vote_g2019
vote_g2018
vote_g2016
vote_g2014
vote_g2012
vote_p2022
vote_p2018
party_reg_state
voter_status
firsttimeseen_voterid
vf_reg_party
vf_reg_pav
registration_date
earliest_registration_date
state_code
tsmart_county_fips
tsmart_census_block_fips
tsmart_census_id_version
housing_temp_address
tsmart_county_name
tsmart_dma_id
tsmart_dma_name
tsmart_cd
tsmart_cd_2010
tsmart_cd_version
tsmart_sd_2020
tsmart_sd_version_2020
tsmart_hd
tsmart_hd_2020
tsmart_hd_2010
tsmart_place_name
zip
vf_reg_county_fips
vf_reg_county_name
vf_reg_dma
vf_reg_dma_name
vf_reg_cd
vf_reg_cd_2020
vf_reg_cd_2010
vf_reg_cd_version_2020
vf_reg_sd
vf_reg_sd_2020
vf_reg_hd_2020
vf_reg_hd_version
vf_reg_hd_version_2020
vf_reg_precinct_id
vf_source_state
vf_reg_zip
vb_email_append_ind
vb_email_append_hh
has_phone
has_wirelessphone
has_email
vb_mailable
coalesced_noncommercial_age
coalesced_noncommercial_ag

In [11]:
print(vf_merged.shape[0])
vf_merged['vote_g2024'] = np.nan
print(vf_merged['vote_g2024'])

1000000
0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
999995   NaN
999996   NaN
999997   NaN
999998   NaN
999999   NaN
Name: vote_g2024, Length: 1000000, dtype: float64


In [22]:
################################################################
#  Relabel columns of vf_merged to be centered around pred_yr
################################################################

# Columns not to adjust
dont_adjust = ['tsmart_cd_2010', 'tsmart_sd_2020', 'tsmart_sd_version_2020', 'tsmart_hd', 'tsmart_hd_2020', 
               'tsmart_hd_2010', 'vf_reg_cd_2020', 'vf_reg_cd_2010', 'vf_reg_cd_version_2020', 'vf_reg_sd_2020', 
               'vf_reg_hd_2020', 'vf_reg_hd_version_2020']

# Function which replaces year with # of years before prediction year
def replace_before_year(col, yr):
    match = re.search(r'(\d{4})$', col)
    if match:
        original_year = int(match.group(1))
        difference = yr - original_year
        # Replace the year with 'yr-X'
        new_col = re.sub(r'\d{4}$', f'yr-{difference}', col)
        return new_col
    return col

# Function which replaces year with # of years after prediction year
def replace_after_year(col, yr):
    match = re.search(r'(\d{4})$', col)
    if match:
        original_year = int(match.group(1))
        diff = original_year - yr
        # Replace the year with 'yr+X'
        new_col = re.sub(r'\d{4}$', f'yr+{diff}', col)
        return new_col
    return col

# Center column names around pred_yr
def vf_year_subset(pred_yr):

    # Define a regex pattern to match column names ending with a four-digit year
    gen_year_pattern = re.compile(r'.*_g\d{4}$')
    prim_year_pattern = re.compile(r'.*_p\d{4}$')

    # Filter columns in vf that match the pattern
    columns_with_gen = [col for col in vf.columns if gen_year_pattern.match(col)]
    columns_with_prim = [col for col in vf.columns if prim_year_pattern.match(col)]

    later_columns = [col for col in columns_with_gen + columns_with_prim if int(col.split('_')[-1][1:]) > pred_yr]
    earlier_columns = [col for col in columns_with_gen + columns_with_prim if int(col.split('_')[-1][1:]) <= pred_yr]

    # vf_year = vf.drop(columns = exclude_columns)
    # print(vf_year.columns)

    vf_year = vf.copy()

    # Rename year-marked columns
    for col in columns_with_gen + columns_with_prim:
        
        if col in later_columns:
            new_col = replace_after_year(col, pred_yr)
            vf_year.rename(columns={col: new_col}, inplace=True)

        if col in earlier_columns:
            new_col = replace_before_year(col, pred_yr)
            vf_year.rename(columns={col: new_col}, inplace=True)

    return vf_year

def vf_year_center(pred_yr):

    cols_to_adjust = list(set(vf_merged.columns) - set(dont_adjust))
    
    year_pattern = re.compile(r'.*\d{4}$')
    year_columns = [col for col in cols_to_adjust if year_pattern.match(col)]

    later_columns = [col for col in year_columns if int(col.split('_')[-1][-4:]) > pred_yr]
    earlier_columns = [col for col in year_columns if int(col.split('_')[-1][-4:]) <= pred_yr]

    vf_year = vf_merged.copy()

    # Rename year-marked columns
    for col in year_columns:
        
        if col in later_columns:
            new_col = replace_after_year(col, pred_yr)
            vf_year.rename(columns={col: new_col}, inplace=True)

        if col in earlier_columns:
            new_col = replace_before_year(col, pred_yr)
            vf_year.rename(columns={col: new_col}, inplace=True)
    
    return vf_year


In [21]:
dont_adjust = ['tsmart_cd_2010', 'tsmart_sd_2020', 'tsmart_sd_version_2020', 'tsmart_hd', 'tsmart_hd_2020', 
               'tsmart_hd_2010', 'vf_reg_cd_2020', 'vf_reg_cd_2010', 'vf_reg_cd_version_2020', 'vf_reg_sd_2020', 
               'vf_reg_hd_2020', 'vf_reg_hd_version_2020']
print(list(set(vf_merged.columns) - set(dont_adjust)))

['tsmart_place_name', 'gender_noncommercial', 'ts_homeowner', 'vote_p2018', 'dvp_cat_2018', 'party_reg_state', 'pwin_2018', 'voter_status', 'po2_2016', 'vf_reg_county_fips', 'dvp_cat_2016', 'state_code', 'inc_2014', 'is_dupe', 'gender_score_is_modeled_noncommercial', 'has_email', 'vote_g2023', 'parent_noncommercial', 'acs_median_hh_income_bucket', 'pwin_2014', 'pres_fr_2012', 'firsttimeseen_voterid', 'home_value_is_null', 'dpres_cat_2022', 'vf_reg_zip', 'vb_email_append_ind', 'marriage_noncommercial', 'household_net_worth_is_null', 'household_income_is_null', 'dexp_cat_gran_2014', 'dpres_cat_2020', 'pwin_2016', 'vb_email_append_hh', 'inc_2020', 'dexp_cat_2020', 'rexp_cat_2020', 'vote_g2022', 'dexp_cat_gran_2012', 'dpres_cat_2014', 'has_phone', 'rexp_cat_2016', 'urbanicity', 'pres_po2_2016', 'pres_inc_2012', 'dexp_cat_2012', 'redist_2022', 'rexp_cat_gran_2022', 'tsmart_cd_version', 'voterbase_id.1', 'inc_2022', 'vf_reg_pav', 'rexp_cat_gran_2020', 'pwin_2020', 'dexp_cat_gran_2018', 'po2_

In [32]:
pred_yr = 2016
vf_2016 = vf_year_center(pred_yr)
# print(vf_2020.head())
for col in vf_2016.columns:
    print(col)

voterbase_id
legal_commercial_model_usage_ok
vb_deceased
vote_gyr+8
vote_gyr+7
vote_gyr+6
vote_gyr+5
vote_gyr+4
vote_gyr+3
vote_gyr+2
vote_gyr-0
vote_gyr-2
vote_gyr-4
vote_pyr+6
vote_pyr+2
party_reg_state
voter_status
firsttimeseen_voterid
vf_reg_party
vf_reg_pav
registration_date
earliest_registration_date
state_code
tsmart_county_fips
tsmart_census_block_fips
tsmart_census_id_version
housing_temp_address
tsmart_county_name
tsmart_dma_id
tsmart_dma_name
tsmart_cd
tsmart_cd_2010
tsmart_cd_version
tsmart_sd_2020
tsmart_sd_version_2020
tsmart_hd
tsmart_hd_2020
tsmart_hd_2010
tsmart_place_name
zip
vf_reg_county_fips
vf_reg_county_name
vf_reg_dma
vf_reg_dma_name
vf_reg_cd
vf_reg_cd_2020
vf_reg_cd_2010
vf_reg_cd_version_2020
vf_reg_sd
vf_reg_sd_2020
vf_reg_hd_2020
vf_reg_hd_version
vf_reg_hd_version_2020
vf_reg_precinct_id
vf_source_state
vf_reg_zip
vb_email_append_ind
vb_email_append_hh
has_phone
has_wirelessphone
has_email
vb_mailable
coalesced_noncommercial_age
coalesced_noncommercial_ag

In [33]:
vf_2016.to_csv('merged_2016_1M.csv', index = False)

In [34]:
print(vf_2016.head())

      voterbase_id  legal_commercial_model_usage_ok  vb_deceased  vote_gyr+8  \
0  MO-000000000272                                1            0         NaN   
1  MO-000000000540                                1            0         NaN   
2  MO-000000000555                                1            0         NaN   
3  MO-000000000558                                1            0         NaN   
4  MO-000000000559                                1            0         NaN   

   vote_gyr+7  vote_gyr+6  vote_gyr+5  vote_gyr+4  vote_gyr+3  vote_gyr+2  \
0           0           1           0           1           0           0   
1           0           1           0           1           0           1   
2           0           1           0           1           0           1   
3           0           1           0           1           0           1   
4           0           1           0           1           0           1   

   ...  pres_po1_yr-0  pres_po2_yr-0  pres_inc_yr+4  pre